In [1]:
import os
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk
from tqdm import tqdm

In [2]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

In [50]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
import re

In [4]:
#changing the directory 
os.chdir("D:/Sravan/DS/NLP/Data Sets/twitter_Sentiment_analysis_AV")

In [5]:
#reading the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [6]:
train.head(3)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...


In [7]:
print(train.shape)
print(test.shape)

(7920, 3)
(1953, 2)


In [8]:
fin_valid = train.tail(20)
fin_train = train.head(7900)

In [9]:
print(fin_valid.shape)
print(fin_train.shape)

(20, 3)
(7900, 3)


In [10]:
#pre_processing function
def raw_to_prepwithtokenize(raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    low_case = letters_only.lower()
    words = nltk.word_tokenize(low_case)
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    other_stop_words = ['.','',' ',"'",',',':']
    meaningful_words = [w for w in words if not w in other_stop_words]
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    processed_sentence = " ".join(meaningful_words)
    return(processed_sentence,len(meaningful_words))

In [11]:
fin_train.columns

Index(['id', 'label', 'tweet'], dtype='object')

In [12]:
num_reviews = fin_train['tweet'].size

In [13]:
num_reviews

7900

In [14]:
fin_nb_words =  0
processed_train_reviews =[]
for i in tqdm(range(0,num_reviews)):
    processed_sentence,nb_words_train = raw_to_prepwithtokenize(fin_train['tweet'][i])
    processed_train_reviews.append(processed_sentence)
    fin_nb_words += nb_words_train

  0%|                                                                                         | 0/7900 [00:00<?, ?it/s]C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|█████████████████████████████████████████████████████████████████████████████| 7900/7900 [00:20<00:00, 386.61it/s]


In [15]:
fin_nb_words

159178

In [16]:
nb_words_train

18

In [15]:
processed_train_reviews[0]

'fingerprint pregnancy test https goo gl h mfqv android apps beautiful cute health igers iphoneonly iphonesia iphone'

In [17]:
fin_valid = fin_valid.reset_index()

In [18]:
fin_valid['tweet']

0     Why did my iPhone calendar link with my Facebo...
1     @brendan_brady1: The new apple advert can fuck...
2     All I wanna do is put music from my iTunes on ...
3     @KaylaHinks @SnapchatProbbz haha so true. Dele...
4     #FuckYou #Apple one thing stops working and I ...
5     ove a fist inside my $&@*# Oh #god #i #a #fist...
6     Legit feels like I have a brand new phone #app...
7     Tipsy rhymes with E, Ye, We, Je, Mi, Ne,... Do...
8     SECOND FUCKING TIME. Stay on hold for 30 mins ...
9     RT: monteithelgort: First post from my new pho...
10    Perfect match #instagood #applewatch #red #ins...
11    I am completely in love with the new iPhone em...
12    Tune in. Turn on. Drop out. & #GTD in one app!...
13    Ok so my galaxy crashed after one day now I ha...
14    Gain Followers RT This MUST FOLLOW ME I FOLLOW...
15    Live out loud #lol #liveoutloud #selfie #smile...
16    We would like to wish you an amazing day! Make...
17    Helping my lovely 90 year old neighbor wit

In [19]:
processed_valid_reviews = []
for i in tqdm(range(0,fin_valid['tweet'].size)):
    processed_sentence,nb_words_valid = raw_to_prepwithtokenize(fin_valid['tweet'][i])
    processed_valid_reviews.append(processed_sentence)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 350.88it/s]


In [41]:
processed_test_reviews = []
for i in tqdm(range(0,test['tweet'].size)):
    processed_sentence,nb_words_test = raw_to_prepwithtokenize(test['tweet'][i])
    processed_test_reviews.append(processed_sentence)


  0%|                                                                                         | 0/1953 [00:00<?, ?it/s]C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))

100%|█████████████████████████████████████████████████████████████████████████████| 1953/1953 [00:05<00:00, 385.35it/s]

In [20]:
processed_train_reviews[1]

'finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias http instagram com p yget jc jm'

In [42]:
processed_test_reviews[1]

'currently shitting my fucking pants apple imac cashmoney raddest swagswagswag http instagr am p uuis bibzo'

In [21]:
len(processed_train_reviews)

7900

In [22]:
NB_WORDS = 10000
max_len = 24 #max number of words in a sequence

In [23]:
tk = Tokenizer(num_words=NB_WORDS,lower=True,split=" ")
tk.fit_on_texts(processed_train_reviews)
train_seq = tk.texts_to_sequences(processed_train_reviews)
valid_seq = tk.texts_to_sequences(processed_valid_reviews)

In [24]:
#padding 
train_seq_pad = pad_sequences(train_seq,maxlen=max_len)
valid_seq_pad = pad_sequences(valid_seq,maxlen=max_len)

In [43]:
test_seq = tk.texts_to_sequences(processed_test_reviews)
#padding
test_seq_pad = pad_sequences(test_seq,maxlen=max_len)

In [25]:
train['label'].unique()

array([0, 1], dtype=int64)

In [26]:
#creting a dictionary of words and the corresponding word embeddings 
emb_dict = {}
glove = open("D:/Sravan/DrQA/Word_Embeddings/glove.840B.300d.txt",encoding='utf-8')
for line in tqdm(glove):
    values = line.split()
    word = values[0]
    other_stop_words = ['.','',' ',"'",',',':','name@domain.com','Killerseats.com']
    vecs = values[1:]
    meaningful_vecs = [w for w in vecs if not w in other_stop_words]
#     raw_vec = values[1:]
#     proc_vec = re.sub("[^0-9]", " ",str(raw_vec))
    vector = np.asarray(meaningful_vecs,dtype='float32')
    emb_dict[word] = vector
glove.close()

1351490it [04:06, 5471.63it/s]

ValueError: could not convert string to float: 'mylot.com'

1351490it [04:20, 5195.64it/s]

In [27]:
fin_nb_words

159178

In [28]:
len(emb_dict)

1352032

In [29]:
len(train_seq_pad[1])

24

In [30]:
#we can also limit the vocabulary count. but here we took a generalized glove embedding lets us use all the words we got.
#create a matrix with NB_Words as rows and dimension as columns
emb_matrix = np.zeros((NB_WORDS,300))
for w ,i in tk.word_index.items():
    if i < NB_WORDS:
        vect = emb_dict.get(w)
        if vect is not None:
            emb_matrix[i] = vect
        else:
            break

# emb_train_reviews = []
# for ind,sent in enumerate(processed_train_reviews):
#     each_review_emb = []
#     if i < len(emb_dict):
#         for word in sent:
#             vect = emb_dict.get(word)
#             if vect is not None:
                
        
        


In [31]:
emb_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [32]:
glove_model = models.Sequential()
glove_model.add(layers.Embedding(NB_WORDS,300,input_length=max_len))
glove_model.add(layers.Flatten())
glove_model.add(layers.Dense(500,activation = 'relu'))
glove_model.add(layers.Dense(500,activation = 'relu'))
glove_model.add(layers.Dense(1,activation='sigmoid'))

In [33]:
glove_model.layers[0].set_weights([emb_matrix])
glove_model.layers[0].trainable = False

In [34]:
glove_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
glove_model.fit(train_seq_pad,fin_train['label'],batch_size=8,epochs=5)

Epoch 1/5


1632/7900 [=====>........................] - ETA: 37:38 - loss: 0.6492 - acc: 0.75 - ETA: 19:29 - loss: 0.5928 - acc: 0.81 - ETA: 13:19 - loss: 0.4652 - acc: 0.87 - ETA: 10:15 - loss: 0.4911 - acc: 0.87 - ETA: 8:24 - loss: 0.6074 - acc: 0.8500 - ETA: 7:10 - loss: 0.7011 - acc: 0.812 - ETA: 6:18 - loss: 0.7136 - acc: 0.785 - ETA: 5:38 - loss: 0.6425 - acc: 0.812 - ETA: 5:07 - loss: 0.6497 - acc: 0.819 - ETA: 4:42 - loss: 0.6738 - acc: 0.812 - ETA: 4:22 - loss: 0.6481 - acc: 0.806 - ETA: 4:05 - loss: 0.6300 - acc: 0.791 - ETA: 3:51 - loss: 0.6112 - acc: 0.788 - ETA: 3:39 - loss: 0.5847 - acc: 0.794 - ETA: 3:28 - loss: 0.5837 - acc: 0.791 - ETA: 3:19 - loss: 0.5737 - acc: 0.796 - ETA: 3:11 - loss: 0.5648 - acc: 0.801 - ETA: 3:04 - loss: 0.5606 - acc: 0.798 - ETA: 2:57 - loss: 0.5455 - acc: 0.809 - ETA: 2:51 - loss: 0.5331 - acc: 0.818 - ETA: 2:46 - loss: 0.5226 - acc: 0.815 - ETA: 2:41 - loss: 0.5136 - acc: 0.818 - ETA: 2:37 - loss: 0.5192 - acc: 0.815 - ETA: 2:33 - loss: 0.5152 - acc: 0.

3344/7900 [===========>..................] - ETA: 1:02 - loss: 0.4267 - acc: 0.825 - ETA: 1:02 - loss: 0.4263 - acc: 0.825 - ETA: 1:02 - loss: 0.4264 - acc: 0.824 - ETA: 1:02 - loss: 0.4245 - acc: 0.825 - ETA: 1:02 - loss: 0.4247 - acc: 0.826 - ETA: 1:02 - loss: 0.4240 - acc: 0.826 - ETA: 1:02 - loss: 0.4252 - acc: 0.825 - ETA: 1:01 - loss: 0.4276 - acc: 0.823 - ETA: 1:01 - loss: 0.4270 - acc: 0.823 - ETA: 1:01 - loss: 0.4289 - acc: 0.823 - ETA: 1:01 - loss: 0.4283 - acc: 0.823 - ETA: 1:01 - loss: 0.4273 - acc: 0.824 - ETA: 1:01 - loss: 0.4259 - acc: 0.825 - ETA: 1:01 - loss: 0.4252 - acc: 0.825 - ETA: 1:01 - loss: 0.4240 - acc: 0.826 - ETA: 1:00 - loss: 0.4234 - acc: 0.826 - ETA: 1:00 - loss: 0.4227 - acc: 0.826 - ETA: 1:00 - loss: 0.4225 - acc: 0.826 - ETA: 1:00 - loss: 0.4218 - acc: 0.826 - ETA: 1:00 - loss: 0.4229 - acc: 0.827 - ETA: 1:00 - loss: 0.4227 - acc: 0.827 - ETA: 1:00 - loss: 0.4246 - acc: 0.828 - ETA: 1:00 - loss: 0.4243 - acc: 0.827 - ETA: 1:00 - loss: 0.4246 - acc: 0.8

5064/7900 [==================>...........] - ETA: 42s - loss: 0.4049 - acc: 0.82 - ETA: 42s - loss: 0.4055 - acc: 0.82 - ETA: 42s - loss: 0.4049 - acc: 0.82 - ETA: 42s - loss: 0.4052 - acc: 0.82 - ETA: 42s - loss: 0.4051 - acc: 0.82 - ETA: 42s - loss: 0.4053 - acc: 0.82 - ETA: 42s - loss: 0.4046 - acc: 0.82 - ETA: 42s - loss: 0.4046 - acc: 0.82 - ETA: 42s - loss: 0.4063 - acc: 0.82 - ETA: 42s - loss: 0.4057 - acc: 0.82 - ETA: 42s - loss: 0.4054 - acc: 0.82 - ETA: 41s - loss: 0.4057 - acc: 0.82 - ETA: 41s - loss: 0.4055 - acc: 0.82 - ETA: 41s - loss: 0.4054 - acc: 0.82 - ETA: 41s - loss: 0.4056 - acc: 0.82 - ETA: 41s - loss: 0.4051 - acc: 0.82 - ETA: 41s - loss: 0.4050 - acc: 0.82 - ETA: 41s - loss: 0.4047 - acc: 0.82 - ETA: 41s - loss: 0.4041 - acc: 0.82 - ETA: 41s - loss: 0.4037 - acc: 0.82 - ETA: 41s - loss: 0.4034 - acc: 0.82 - ETA: 41s - loss: 0.4029 - acc: 0.82 - ETA: 41s - loss: 0.4029 - acc: 0.82 - ETA: 40s - loss: 0.4024 - acc: 0.82 - ETA: 40s - loss: 0.4035 - acc: 0.82 - ETA: 

6784/7900 [========================>.....] - ETA: 26s - loss: 0.3760 - acc: 0.84 - ETA: 26s - loss: 0.3757 - acc: 0.84 - ETA: 26s - loss: 0.3759 - acc: 0.84 - ETA: 25s - loss: 0.3755 - acc: 0.84 - ETA: 25s - loss: 0.3757 - acc: 0.84 - ETA: 25s - loss: 0.3760 - acc: 0.84 - ETA: 25s - loss: 0.3758 - acc: 0.84 - ETA: 25s - loss: 0.3758 - acc: 0.84 - ETA: 25s - loss: 0.3757 - acc: 0.84 - ETA: 25s - loss: 0.3758 - acc: 0.84 - ETA: 25s - loss: 0.3754 - acc: 0.84 - ETA: 25s - loss: 0.3753 - acc: 0.84 - ETA: 25s - loss: 0.3755 - acc: 0.84 - ETA: 25s - loss: 0.3755 - acc: 0.84 - ETA: 25s - loss: 0.3752 - acc: 0.84 - ETA: 24s - loss: 0.3754 - acc: 0.84 - ETA: 24s - loss: 0.3754 - acc: 0.84 - ETA: 24s - loss: 0.3752 - acc: 0.84 - ETA: 24s - loss: 0.3754 - acc: 0.84 - ETA: 24s - loss: 0.3751 - acc: 0.84 - ETA: 24s - loss: 0.3749 - acc: 0.84 - ETA: 24s - loss: 0.3749 - acc: 0.84 - ETA: 24s - loss: 0.3746 - acc: 0.84 - ETA: 24s - loss: 0.3743 - acc: 0.84 - ETA: 24s - loss: 0.3745 - acc: 0.84 - ETA: 

...

1688/7900 [=====>........................] - ETA: 58s - loss: 0.4153 - acc: 0.62 - ETA: 1:00 - loss: 0.3102 - acc: 0.750 - ETA: 1:02 - loss: 0.2328 - acc: 0.833 - ETA: 1:02 - loss: 0.2018 - acc: 0.875 - ETA: 1:03 - loss: 0.1652 - acc: 0.900 - ETA: 1:02 - loss: 0.1498 - acc: 0.916 - ETA: 1:02 - loss: 0.1365 - acc: 0.928 - ETA: 1:02 - loss: 0.1204 - acc: 0.937 - ETA: 1:02 - loss: 0.2206 - acc: 0.930 - ETA: 1:02 - loss: 0.2316 - acc: 0.912 - ETA: 1:02 - loss: 0.2211 - acc: 0.920 - ETA: 1:02 - loss: 0.2152 - acc: 0.916 - ETA: 1:02 - loss: 0.2027 - acc: 0.923 - ETA: 1:02 - loss: 0.1941 - acc: 0.928 - ETA: 1:02 - loss: 0.1877 - acc: 0.933 - ETA: 1:02 - loss: 0.1926 - acc: 0.929 - ETA: 1:02 - loss: 0.1871 - acc: 0.933 - ETA: 1:02 - loss: 0.1796 - acc: 0.937 - ETA: 1:02 - loss: 0.1742 - acc: 0.940 - ETA: 1:02 - loss: 0.1731 - acc: 0.937 - ETA: 1:02 - loss: 0.1725 - acc: 0.934 - ETA: 1:02 - loss: 0.1655 - acc: 0.937 - ETA: 1:02 - loss: 0.1650 - acc: 0.934 - ETA: 1:02 - loss: 0.1703 - acc: 0.932

3408/7900 [===========>..................] - ETA: 52s - loss: 0.1253 - acc: 0.94 - ETA: 52s - loss: 0.1274 - acc: 0.94 - ETA: 52s - loss: 0.1272 - acc: 0.94 - ETA: 52s - loss: 0.1270 - acc: 0.94 - ETA: 52s - loss: 0.1265 - acc: 0.94 - ETA: 52s - loss: 0.1271 - acc: 0.94 - ETA: 52s - loss: 0.1269 - acc: 0.94 - ETA: 52s - loss: 0.1264 - acc: 0.94 - ETA: 52s - loss: 0.1276 - acc: 0.94 - ETA: 52s - loss: 0.1281 - acc: 0.94 - ETA: 52s - loss: 0.1277 - acc: 0.94 - ETA: 52s - loss: 0.1280 - acc: 0.94 - ETA: 51s - loss: 0.1276 - acc: 0.94 - ETA: 51s - loss: 0.1276 - acc: 0.94 - ETA: 51s - loss: 0.1282 - acc: 0.94 - ETA: 51s - loss: 0.1299 - acc: 0.94 - ETA: 51s - loss: 0.1307 - acc: 0.94 - ETA: 51s - loss: 0.1301 - acc: 0.94 - ETA: 51s - loss: 0.1296 - acc: 0.94 - ETA: 51s - loss: 0.1295 - acc: 0.94 - ETA: 51s - loss: 0.1297 - acc: 0.94 - ETA: 51s - loss: 0.1293 - acc: 0.94 - ETA: 51s - loss: 0.1302 - acc: 0.94 - ETA: 51s - loss: 0.1297 - acc: 0.94 - ETA: 51s - loss: 0.1301 - acc: 0.94 - ETA: 

5128/7900 [==================>...........] - ETA: 37s - loss: 0.1220 - acc: 0.94 - ETA: 37s - loss: 0.1218 - acc: 0.94 - ETA: 37s - loss: 0.1215 - acc: 0.94 - ETA: 37s - loss: 0.1224 - acc: 0.94 - ETA: 37s - loss: 0.1221 - acc: 0.94 - ETA: 37s - loss: 0.1219 - acc: 0.94 - ETA: 37s - loss: 0.1220 - acc: 0.94 - ETA: 37s - loss: 0.1217 - acc: 0.94 - ETA: 37s - loss: 0.1216 - acc: 0.94 - ETA: 37s - loss: 0.1226 - acc: 0.94 - ETA: 37s - loss: 0.1232 - acc: 0.94 - ETA: 37s - loss: 0.1229 - acc: 0.94 - ETA: 37s - loss: 0.1229 - acc: 0.94 - ETA: 37s - loss: 0.1227 - acc: 0.94 - ETA: 36s - loss: 0.1225 - acc: 0.94 - ETA: 36s - loss: 0.1225 - acc: 0.94 - ETA: 36s - loss: 0.1224 - acc: 0.94 - ETA: 36s - loss: 0.1227 - acc: 0.94 - ETA: 36s - loss: 0.1232 - acc: 0.94 - ETA: 36s - loss: 0.1229 - acc: 0.94 - ETA: 36s - loss: 0.1234 - acc: 0.94 - ETA: 36s - loss: 0.1233 - acc: 0.94 - ETA: 36s - loss: 0.1231 - acc: 0.94 - ETA: 36s - loss: 0.1230 - acc: 0.94 - ETA: 36s - loss: 0.1229 - acc: 0.94 - ETA: 

6848/7900 [=========================>....] - ETA: 23s - loss: 0.1322 - acc: 0.94 - ETA: 23s - loss: 0.1325 - acc: 0.94 - ETA: 23s - loss: 0.1327 - acc: 0.94 - ETA: 23s - loss: 0.1327 - acc: 0.94 - ETA: 23s - loss: 0.1326 - acc: 0.94 - ETA: 22s - loss: 0.1327 - acc: 0.94 - ETA: 22s - loss: 0.1328 - acc: 0.94 - ETA: 22s - loss: 0.1332 - acc: 0.94 - ETA: 22s - loss: 0.1331 - acc: 0.94 - ETA: 22s - loss: 0.1329 - acc: 0.94 - ETA: 22s - loss: 0.1330 - acc: 0.94 - ETA: 22s - loss: 0.1331 - acc: 0.94 - ETA: 22s - loss: 0.1331 - acc: 0.94 - ETA: 22s - loss: 0.1334 - acc: 0.94 - ETA: 22s - loss: 0.1333 - acc: 0.94 - ETA: 22s - loss: 0.1333 - acc: 0.94 - ETA: 22s - loss: 0.1332 - acc: 0.94 - ETA: 22s - loss: 0.1330 - acc: 0.94 - ETA: 22s - loss: 0.1329 - acc: 0.94 - ETA: 22s - loss: 0.1328 - acc: 0.94 - ETA: 21s - loss: 0.1328 - acc: 0.94 - ETA: 21s - loss: 0.1333 - acc: 0.94 - ETA: 21s - loss: 0.1331 - acc: 0.94 - ETA: 21s - loss: 0.1330 - acc: 0.94 - ETA: 21s - loss: 0.1329 - acc: 0.94 - ETA: 

7900/7900 [==============================] - ETA: 8s - loss: 0.1327 - acc: 0.943 - ETA: 8s - loss: 0.1326 - acc: 0.943 - ETA: 8s - loss: 0.1324 - acc: 0.943 - ETA: 8s - loss: 0.1324 - acc: 0.943 - ETA: 8s - loss: 0.1323 - acc: 0.943 - ETA: 8s - loss: 0.1322 - acc: 0.943 - ETA: 8s - loss: 0.1327 - acc: 0.943 - ETA: 8s - loss: 0.1326 - acc: 0.943 - ETA: 8s - loss: 0.1325 - acc: 0.943 - ETA: 8s - loss: 0.1325 - acc: 0.943 - ETA: 8s - loss: 0.1333 - acc: 0.943 - ETA: 8s - loss: 0.1332 - acc: 0.943 - ETA: 8s - loss: 0.1335 - acc: 0.943 - ETA: 7s - loss: 0.1336 - acc: 0.943 - ETA: 7s - loss: 0.1335 - acc: 0.943 - ETA: 7s - loss: 0.1335 - acc: 0.943 - ETA: 7s - loss: 0.1334 - acc: 0.943 - ETA: 7s - loss: 0.1332 - acc: 0.943 - ETA: 7s - loss: 0.1331 - acc: 0.943 - ETA: 7s - loss: 0.1331 - acc: 0.943 - ETA: 7s - loss: 0.1330 - acc: 0.943 - ETA: 7s - loss: 0.1331 - acc: 0.943 - ETA: 7s - loss: 0.1330 - acc: 0.943 - ETA: 7s - loss: 0.1329 - acc: 0.943 - ETA: 7s - loss: 0.1329 - acc: 0.943 - ETA: 

In [36]:
pred_valid = glove_model.predict_classes(valid_seq_pad)

In [37]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(fin_valid['label'],pred_valid)

0.8

In [44]:
#test prediction
test_pred = glove_model.predict_classes(test_seq_pad)

In [47]:
test_pred[:,0]

array([1, 0, 1, ..., 1, 1, 0])

In [49]:
output = pd.DataFrame(data={"id":test["id"], "label":test_pred[:,0]})
#Use pandas to write the comma-separated output file
output.to_csv( "twitter_sentiment_analysis1.csv", index=False, quoting=3)

In [57]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(fin_train['label'])

In [70]:
y_valid = to_categorical(fin_valid['label'])

In [59]:
y_train.shape

(7900, 2)

In [60]:
train_seq_pad.shape

(7900, 24)

In [65]:
#LSTM:
lstm_out = 196
lstm_model = models.Sequential()
lstm_model.add(layers.Embedding(NB_WORDS,300,input_length=max_len))
lstm_model.add(layers.SpatialDropout1D(0.4))
lstm_model.add(layers.LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2))
lstm_model.add(layers.Dense(2,activation='sigmoid'))

In [66]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 24, 300)           3000000   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 24, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               389648    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 394       
Total params: 3,390,042
Trainable params: 3,390,042
Non-trainable params: 0
_________________________________________________________________


In [68]:
lstm_model.compile(optimizer = 'adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [69]:
lstm_model.fit(train_seq_pad,y_train,batch_size=8,epochs=5)

Epoch 1/5


1632/7900 [=====>........................] - ETA: 30:50 - loss: 0.6914 - acc: 0.50 - ETA: 16:03 - loss: 0.6900 - acc: 0.62 - ETA: 11:09 - loss: 0.6865 - acc: 0.66 - ETA: 8:42 - loss: 0.6828 - acc: 0.6875 - ETA: 7:12 - loss: 0.6851 - acc: 0.625 - ETA: 6:13 - loss: 0.6822 - acc: 0.645 - ETA: 5:30 - loss: 0.6753 - acc: 0.678 - ETA: 4:58 - loss: 0.6735 - acc: 0.687 - ETA: 4:33 - loss: 0.6673 - acc: 0.694 - ETA: 4:13 - loss: 0.6607 - acc: 0.700 - ETA: 3:57 - loss: 0.6577 - acc: 0.693 - ETA: 3:43 - loss: 0.6645 - acc: 0.666 - ETA: 3:32 - loss: 0.6536 - acc: 0.682 - ETA: 3:22 - loss: 0.6445 - acc: 0.687 - ETA: 3:13 - loss: 0.6495 - acc: 0.675 - ETA: 3:06 - loss: 0.6379 - acc: 0.687 - ETA: 2:59 - loss: 0.6281 - acc: 0.691 - ETA: 2:53 - loss: 0.6226 - acc: 0.694 - ETA: 2:48 - loss: 0.6193 - acc: 0.690 - ETA: 2:43 - loss: 0.6065 - acc: 0.700 - ETA: 2:39 - loss: 0.5851 - acc: 0.714 - ETA: 2:35 - loss: 0.5618 - acc: 0.727 - ETA: 2:31 - loss: 0.6048 - acc: 0.717 - ETA: 2:28 - loss: 0.5961 - acc: 0.

3312/7900 [===========>..................] - ETA: 1:11 - loss: 0.3559 - acc: 0.843 - ETA: 1:10 - loss: 0.3549 - acc: 0.843 - ETA: 1:10 - loss: 0.3547 - acc: 0.844 - ETA: 1:10 - loss: 0.3551 - acc: 0.843 - ETA: 1:10 - loss: 0.3562 - acc: 0.843 - ETA: 1:10 - loss: 0.3554 - acc: 0.844 - ETA: 1:10 - loss: 0.3547 - acc: 0.844 - ETA: 1:10 - loss: 0.3537 - acc: 0.844 - ETA: 1:10 - loss: 0.3530 - acc: 0.845 - ETA: 1:09 - loss: 0.3521 - acc: 0.845 - ETA: 1:09 - loss: 0.3522 - acc: 0.845 - ETA: 1:09 - loss: 0.3524 - acc: 0.844 - ETA: 1:09 - loss: 0.3526 - acc: 0.844 - ETA: 1:09 - loss: 0.3514 - acc: 0.845 - ETA: 1:09 - loss: 0.3514 - acc: 0.845 - ETA: 1:09 - loss: 0.3510 - acc: 0.845 - ETA: 1:09 - loss: 0.3503 - acc: 0.845 - ETA: 1:08 - loss: 0.3495 - acc: 0.845 - ETA: 1:08 - loss: 0.3487 - acc: 0.845 - ETA: 1:08 - loss: 0.3494 - acc: 0.845 - ETA: 1:08 - loss: 0.3497 - acc: 0.845 - ETA: 1:08 - loss: 0.3485 - acc: 0.845 - ETA: 1:08 - loss: 0.3472 - acc: 0.846 - ETA: 1:08 - loss: 0.3462 - acc: 0.8

5032/7900 [==================>...........] - ETA: 48s - loss: 0.3132 - acc: 0.86 - ETA: 48s - loss: 0.3132 - acc: 0.86 - ETA: 48s - loss: 0.3135 - acc: 0.86 - ETA: 48s - loss: 0.3134 - acc: 0.86 - ETA: 48s - loss: 0.3130 - acc: 0.86 - ETA: 48s - loss: 0.3125 - acc: 0.86 - ETA: 47s - loss: 0.3123 - acc: 0.86 - ETA: 47s - loss: 0.3120 - acc: 0.86 - ETA: 47s - loss: 0.3114 - acc: 0.86 - ETA: 47s - loss: 0.3119 - acc: 0.86 - ETA: 47s - loss: 0.3120 - acc: 0.86 - ETA: 47s - loss: 0.3117 - acc: 0.86 - ETA: 47s - loss: 0.3115 - acc: 0.86 - ETA: 47s - loss: 0.3116 - acc: 0.86 - ETA: 47s - loss: 0.3122 - acc: 0.86 - ETA: 47s - loss: 0.3121 - acc: 0.86 - ETA: 47s - loss: 0.3120 - acc: 0.86 - ETA: 46s - loss: 0.3119 - acc: 0.86 - ETA: 46s - loss: 0.3118 - acc: 0.86 - ETA: 46s - loss: 0.3128 - acc: 0.86 - ETA: 46s - loss: 0.3129 - acc: 0.86 - ETA: 46s - loss: 0.3122 - acc: 0.86 - ETA: 46s - loss: 0.3124 - acc: 0.86 - ETA: 46s - loss: 0.3130 - acc: 0.86 - ETA: 46s - loss: 0.3123 - acc: 0.86 - ETA: 

6752/7900 [========================>.....] - ETA: 29s - loss: 0.2941 - acc: 0.87 - ETA: 29s - loss: 0.2937 - acc: 0.87 - ETA: 29s - loss: 0.2944 - acc: 0.87 - ETA: 29s - loss: 0.2941 - acc: 0.87 - ETA: 29s - loss: 0.2937 - acc: 0.87 - ETA: 29s - loss: 0.2941 - acc: 0.87 - ETA: 29s - loss: 0.2950 - acc: 0.87 - ETA: 29s - loss: 0.2953 - acc: 0.87 - ETA: 28s - loss: 0.2951 - acc: 0.87 - ETA: 28s - loss: 0.2954 - acc: 0.87 - ETA: 28s - loss: 0.2951 - acc: 0.87 - ETA: 28s - loss: 0.2949 - acc: 0.87 - ETA: 28s - loss: 0.2958 - acc: 0.87 - ETA: 28s - loss: 0.2959 - acc: 0.87 - ETA: 28s - loss: 0.2956 - acc: 0.87 - ETA: 28s - loss: 0.2956 - acc: 0.87 - ETA: 28s - loss: 0.2957 - acc: 0.87 - ETA: 28s - loss: 0.2954 - acc: 0.87 - ETA: 28s - loss: 0.2952 - acc: 0.87 - ETA: 28s - loss: 0.2951 - acc: 0.87 - ETA: 28s - loss: 0.2950 - acc: 0.87 - ETA: 27s - loss: 0.2949 - acc: 0.87 - ETA: 27s - loss: 0.2947 - acc: 0.87 - ETA: 27s - loss: 0.2945 - acc: 0.87 - ETA: 27s - loss: 0.2947 - acc: 0.87 - ETA: 

...

1632/7900 [=====>........................] - ETA: 1:08 - loss: 0.0146 - acc: 1.000 - ETA: 1:08 - loss: 0.0361 - acc: 1.000 - ETA: 1:09 - loss: 0.0272 - acc: 1.000 - ETA: 1:10 - loss: 0.0210 - acc: 1.000 - ETA: 1:10 - loss: 0.0204 - acc: 1.000 - ETA: 1:10 - loss: 0.0198 - acc: 1.000 - ETA: 1:10 - loss: 0.0186 - acc: 1.000 - ETA: 1:10 - loss: 0.0171 - acc: 1.000 - ETA: 1:10 - loss: 0.0385 - acc: 0.986 - ETA: 1:11 - loss: 0.0348 - acc: 0.987 - ETA: 1:11 - loss: 0.0338 - acc: 0.988 - ETA: 1:11 - loss: 0.0325 - acc: 0.989 - ETA: 1:11 - loss: 0.0311 - acc: 0.990 - ETA: 1:10 - loss: 0.0304 - acc: 0.991 - ETA: 1:11 - loss: 0.0507 - acc: 0.983 - ETA: 1:11 - loss: 0.0477 - acc: 0.984 - ETA: 1:11 - loss: 0.0459 - acc: 0.985 - ETA: 1:11 - loss: 0.0459 - acc: 0.986 - ETA: 1:11 - loss: 0.0442 - acc: 0.986 - ETA: 1:11 - loss: 0.0421 - acc: 0.987 - ETA: 1:11 - loss: 0.0433 - acc: 0.988 - ETA: 1:11 - loss: 0.0451 - acc: 0.988 - ETA: 1:11 - loss: 0.0521 - acc: 0.983 - ETA: 1:11 - loss: 0.0504 - acc: 0.9

3352/7900 [===========>..................] - ETA: 1:00 - loss: 0.0587 - acc: 0.978 - ETA: 1:00 - loss: 0.0602 - acc: 0.977 - ETA: 1:00 - loss: 0.0608 - acc: 0.977 - ETA: 59s - loss: 0.0605 - acc: 0.977 - ETA: 59s - loss: 0.0603 - acc: 0.97 - ETA: 59s - loss: 0.0614 - acc: 0.97 - ETA: 59s - loss: 0.0613 - acc: 0.97 - ETA: 59s - loss: 0.0610 - acc: 0.97 - ETA: 59s - loss: 0.0609 - acc: 0.97 - ETA: 59s - loss: 0.0610 - acc: 0.97 - ETA: 59s - loss: 0.0607 - acc: 0.97 - ETA: 59s - loss: 0.0605 - acc: 0.97 - ETA: 59s - loss: 0.0607 - acc: 0.97 - ETA: 59s - loss: 0.0604 - acc: 0.97 - ETA: 59s - loss: 0.0606 - acc: 0.97 - ETA: 59s - loss: 0.0603 - acc: 0.97 - ETA: 58s - loss: 0.0601 - acc: 0.97 - ETA: 58s - loss: 0.0602 - acc: 0.97 - ETA: 58s - loss: 0.0602 - acc: 0.97 - ETA: 58s - loss: 0.0600 - acc: 0.97 - ETA: 58s - loss: 0.0604 - acc: 0.97 - ETA: 58s - loss: 0.0601 - acc: 0.97 - ETA: 58s - loss: 0.0600 - acc: 0.97 - ETA: 58s - loss: 0.0599 - acc: 0.97 - ETA: 58s - loss: 0.0607 - acc: 0.97 

5072/7900 [==================>...........] - ETA: 44s - loss: 0.0591 - acc: 0.97 - ETA: 44s - loss: 0.0594 - acc: 0.97 - ETA: 43s - loss: 0.0593 - acc: 0.97 - ETA: 43s - loss: 0.0592 - acc: 0.97 - ETA: 43s - loss: 0.0590 - acc: 0.97 - ETA: 43s - loss: 0.0589 - acc: 0.97 - ETA: 43s - loss: 0.0590 - acc: 0.97 - ETA: 43s - loss: 0.0589 - acc: 0.97 - ETA: 43s - loss: 0.0588 - acc: 0.97 - ETA: 43s - loss: 0.0586 - acc: 0.97 - ETA: 43s - loss: 0.0586 - acc: 0.97 - ETA: 43s - loss: 0.0585 - acc: 0.97 - ETA: 43s - loss: 0.0584 - acc: 0.97 - ETA: 43s - loss: 0.0583 - acc: 0.97 - ETA: 42s - loss: 0.0582 - acc: 0.97 - ETA: 42s - loss: 0.0581 - acc: 0.97 - ETA: 42s - loss: 0.0581 - acc: 0.97 - ETA: 42s - loss: 0.0586 - acc: 0.97 - ETA: 42s - loss: 0.0585 - acc: 0.97 - ETA: 42s - loss: 0.0585 - acc: 0.97 - ETA: 42s - loss: 0.0584 - acc: 0.97 - ETA: 42s - loss: 0.0585 - acc: 0.97 - ETA: 42s - loss: 0.0583 - acc: 0.97 - ETA: 42s - loss: 0.0582 - acc: 0.97 - ETA: 42s - loss: 0.0581 - acc: 0.97 - ETA: 

6792/7900 [========================>.....] - ETA: 27s - loss: 0.0591 - acc: 0.97 - ETA: 27s - loss: 0.0590 - acc: 0.97 - ETA: 27s - loss: 0.0590 - acc: 0.97 - ETA: 27s - loss: 0.0589 - acc: 0.97 - ETA: 27s - loss: 0.0589 - acc: 0.97 - ETA: 26s - loss: 0.0589 - acc: 0.97 - ETA: 26s - loss: 0.0588 - acc: 0.97 - ETA: 26s - loss: 0.0588 - acc: 0.97 - ETA: 26s - loss: 0.0590 - acc: 0.97 - ETA: 26s - loss: 0.0589 - acc: 0.97 - ETA: 26s - loss: 0.0588 - acc: 0.97 - ETA: 26s - loss: 0.0589 - acc: 0.97 - ETA: 26s - loss: 0.0588 - acc: 0.97 - ETA: 26s - loss: 0.0587 - acc: 0.97 - ETA: 26s - loss: 0.0587 - acc: 0.97 - ETA: 26s - loss: 0.0586 - acc: 0.97 - ETA: 26s - loss: 0.0585 - acc: 0.97 - ETA: 26s - loss: 0.0587 - acc: 0.97 - ETA: 26s - loss: 0.0587 - acc: 0.97 - ETA: 25s - loss: 0.0586 - acc: 0.97 - ETA: 25s - loss: 0.0585 - acc: 0.97 - ETA: 25s - loss: 0.0584 - acc: 0.97 - ETA: 25s - loss: 0.0584 - acc: 0.97 - ETA: 25s - loss: 0.0583 - acc: 0.97 - ETA: 25s - loss: 0.0582 - acc: 0.97 - ETA: 

7900/7900 [==============================] - ETA: 10s - loss: 0.0643 - acc: 0.97 - ETA: 10s - loss: 0.0645 - acc: 0.97 - ETA: 10s - loss: 0.0645 - acc: 0.97 - ETA: 10s - loss: 0.0645 - acc: 0.97 - ETA: 10s - loss: 0.0645 - acc: 0.97 - ETA: 10s - loss: 0.0644 - acc: 0.97 - ETA: 10s - loss: 0.0644 - acc: 0.97 - ETA: 10s - loss: 0.0646 - acc: 0.97 - ETA: 10s - loss: 0.0645 - acc: 0.97 - ETA: 9s - loss: 0.0645 - acc: 0.9763 - ETA: 9s - loss: 0.0644 - acc: 0.976 - ETA: 9s - loss: 0.0644 - acc: 0.976 - ETA: 9s - loss: 0.0644 - acc: 0.976 - ETA: 9s - loss: 0.0643 - acc: 0.976 - ETA: 9s - loss: 0.0643 - acc: 0.976 - ETA: 9s - loss: 0.0645 - acc: 0.976 - ETA: 9s - loss: 0.0648 - acc: 0.976 - ETA: 9s - loss: 0.0649 - acc: 0.976 - ETA: 9s - loss: 0.0648 - acc: 0.976 - ETA: 9s - loss: 0.0648 - acc: 0.976 - ETA: 9s - loss: 0.0654 - acc: 0.975 - ETA: 9s - loss: 0.0654 - acc: 0.975 - ETA: 8s - loss: 0.0653 - acc: 0.975 - ETA: 8s - loss: 0.0653 - acc: 0.975 - ETA: 8s - loss: 0.0652 - acc: 0.976 - ETA:

In [80]:
pred_valid_lstm=lstm_model.predict_classes(valid_seq_pad)

In [81]:
pred_valid_lstm

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [82]:
pred_valid_lstm.shape

(20,)

In [77]:
fin_valid['label'].shape

(20,)

In [83]:
accuracy_score(fin_valid['label'],pred_valid_lstm)

0.8

In [84]:
test_pred_lstm = lstm_model.predict_classes(test_seq_pad)

In [87]:
output = pd.DataFrame(data={"id":test["id"], "label":test_pred_lstm})
#Use pandas to write the comma-separated output file
output.to_csv( "twitter_sentiment_analysis2.csv", index=False, quoting=3)